# LIF to TIF

This notebook uses the library `readlif` from the package [AICSImage](https://allencellmodeling.github.io/aicsimageio/) to read the microscopy Leica Image Files (LIFs) to then convert the files to 'tif' format, which is better handled by Python.

Some useful functions in AICSImage:
* `img.data` returns 5D TCZYX numpy array
* `.shape` returns tuple of dimension sizes in TCZYX order
* `img.dims`. The order is: "T, C, Z, Y, X"
* `metadata` returns the metadata object for this file format

# Example data

Live (`LIF01`, `LIF03`) and fixed-tissue (`LIF02`) confocal images. 


# Import the libraries 

Double-check that all the libraries listed below are installed.

You may need to install: `pip install readlif` AND `pip install bioformats_jar`

In [ ]:
import numpy as np

import re
import os
import sys
import pandas as pd

from aicsimageio import AICSImage
import tifffile

import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString

# Create the paths


Define the **'notebook_name'** (i.e. your project name) and the **'data_path'** names to create all the necessary folders for your project. 

In [ ]:
notebook_name = 'lif_to_tif'

# Data path to 'Data_example' folders. Change accordingly to your data structure.
data_path = os.path.dirname(os.getcwd())  # Moves one level up from the current directory

# Change the folder names accordingly
paths = {'data': data_path,
         'raw_data':  f'{data_path}/Data_examples/{notebook_name}/',
         'processed_data': f'{data_path}/Processed_data_examples/{notebook_name}/',
         'analysis': f'{data_path}/Analysis_examples/{notebook_name}/',         
         'plots': f'{data_path}/Analysis_examples/{notebook_name}/Plots/'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Define the functions

Here are some function templates that you can modify and expand to your needs. For example, to export specific metadata, change the paths, or save more than 2 individual channels.

In [ ]:
def lif_to_hyperstack(stack, filename):
    stack_transposed = np.transpose(stack.data, (0, 2, 1, 3, 4))
    save_path = f"{paths['processed_data']}{os.path.splitext(filename)[0]}.tif"
    tifffile.imwrite(save_path, stack_transposed, imagej=True)
    print(save_path)

In [ ]:
def lif_metadata(stack, filename):
    # Create the metadata directory 
    metadata_dir = f"{paths['processed_data']}{'metadata'}"
    os.makedirs(metadata_dir, exist_ok=True)
    
    # Convert metadata to XML
    metadata_str = parseString(ET.tostring(stack.metadata)).toprettyxml(indent="\t")
    xml_file_path = os.path.join(metadata_dir, f"{os.path.splitext(filename)[0]}_metadata.xml")
    
    # Write the metadata to the XML file
    with open(xml_file_path, 'w') as xml_file:
        xml_file.write(metadata_str)


In [ ]:
def lif_to_channels(stack, filename, max_projection=False):

    # Time-series and two channels:
    if stack.dims.T > 1 and stack.dims.C == 2:
        ch1 = stack.data[:, 0, :, :, :]
        ch2 = stack.data[:, 1, :, :, :]

        # Time-series, two-channels and z-stack:
        if max_projection and stack.dims.Z > 1:
            ch1 = np.max(ch1, axis=1)
            ch2 = np.max(ch2, axis=1)
            ch1_suffix = "_ch1_z-max.tif"
            ch2_suffix = "_ch2_z-max.tif"
        else:
            ch1_suffix = "_ch1.tif"
            ch2_suffix = "_ch2.tif"
        
        # Save one TIFF per channel
        save_path_ch1 = f"{paths['processed_data']}{os.path.splitext(filename)[0]}{ch1_suffix}"
        save_path_ch2 = f"{paths['processed_data']}{os.path.splitext(filename)[0]}{ch2_suffix}"
        
        tifffile.imwrite(save_path_ch1, ch1)
        tifffile.imwrite(save_path_ch2, ch2)
        
        print(save_path_ch1), print(save_path_ch2)

# Save Leica files as hyperstacks

## Loop through all folders

You can use `os.walk()` to go through the file names in a directory tree by walking the tree either top-down or bottom-up. For each directory in the tree rooted at directory `top` (including `top` itself), it yields a 3-tuple: `dirpath`, `dirnames`, `filenames`. See [Documentation](https://docs.python.org/3/library/os.html).

Additionally, you can use **if** statements to look for specific filenames or file types:
- Regular expressions: `re.match()` checks for a match only at the beginning of the string, while `re.search()` checks for a match anywhere in the string. See [Documentation](https://docs.python.org/3/library/re.html) and [cheat sheet](https://cheatography.com/davechild/cheat-sheets/regular-expressions/).
- Python built-in functions for text or [strings](https://python-reference.readthedocs.io/en/latest/docs/str/): `startswith`, `endswith`, etc. See [Documentation](https://docs.python.org/3/library/stdtypes.html#text-sequence-type-str).

**Note**: The example data includes time-series, time-series with z-stacks, and multi-channel z-stacks.

In [ ]:
lif_files_found = False

# Walk through the directory
for root, subdirs, files in os.walk(paths['raw_data']):
    for file in files:
        if file.endswith('.lif'):  
            lif_files_found = True  # Set the flag to True when a .lif file is found

            # Load the .lif file using AICSImage
            stack = AICSImage(os.path.join(root, file))
    
            # Save LIF as hyperstack and LIF metadata
            lif_to_hyperstack(stack, file)
            lif_metadata(stack, file)

# Print statement if no 'lif' files were found
if not lif_files_found:
    print("No '.lif' files were found")

## Loop through specific recordings

This is another way to loop through specific 'recordings' saved in an specific location ('recordings_path'). If the file is not found, it will print the file path of the file not found.

In [ ]:
recordings_path = paths['raw_data']

recordings = ['LIF01', 'LIF03']

for recording in recordings:
    file_path = f"{recordings_path}{recording}.lif"

    try:
        # Load the .lif file using AICSImage
        stack = AICSImage(file_path)

        # Save LIF as hyperstack and LIF metadata
        lif_to_hyperstack(stack, recording)
        lif_metadata(stack, recording)
    
    except FileNotFoundError:
        print(f"File not found: {file_path}")

# Save 'LIF' files as 'TIF' files

An example of how to combine different functions to save the LIF files as hyperstacks or single files as needed.

Print statements can be substituted with [logging](https://docs.python.org/3/library/logging.html) for better control.

In [ ]:
# Walk through the directory
for root, subdirs, files in os.walk(paths['raw_data']):
    for file in files:
        if file.endswith('.lif'):  # Define Leica files
           
            # Load the .lif file using AICSImage
            stack = AICSImage(os.path.join(root, file))

            # Time-series with 2 channels and z-stacks
            if stack.dims.T > 1 and stack.dims.C > 1:
                lif_to_channels(stack, file, max_projection=False)
                # Save the metadata
                lif_metadata(stack, file)

            else:
                # Save LIF as hyperstack and LIF metadata
                lif_to_hyperstack(stack, file)
                lif_metadata(stack, file)